<a href="https://colab.research.google.com/github/serimleee/analysis/blob/main/Line_Ads_Data_Scientiest_%EC%9D%B4%EC%84%B8%EB%A6%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

%config InlineBackend.figure_format = 'retina' # 선명하게 한글이 보일 수 있게
!apt -qq -y install fonts-nanum # 나눔 설치

import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
%config InlineBackend.figure_format = 'retina'
plt.rc('font', family='NanumBarunGothic')

In [ ]:
# 데이터업로드
from google.colab import files
uploaded = files.upload()

In [ ]:
users = pd.read_csv('_users.csv')
purchase_log = pd.read_csv('_purchase_log.csv')
products = pd.read_csv('_products.csv', encoding = 'cp949')
click_log = pd.read_csv('_click_log.csv')
_20230401_users = pd.read_csv('_20230401_users.csv')

In [ ]:
# 사용자 연령,성별 정보
users.head(10)

In [ ]:
# 사용자의 상품 구매이력
purchase_log.head(10)

In [ ]:
# 상품의 메타 정보
products.head(10)

In [ ]:
# 사용자의 상품 클릭이력
click_log.head(10)

In [ ]:
# 추천 대상 사용자 정보
_20230401_users.head(10)

In [ ]:
# 각각의 데이터 건수
print('users :' + str(len(users)), '\n'
      , 'purchase_log :' + str(len(purchase_log)), '\n'
      , 'click_log :' + str(len(click_log)), '\n'
      , 'products :' + str(len(products)), '\n'
      , '_20230401_users :' + str(len(_20230401_users)))

# 상품 추천을 위한 Insight 분석

## 1) 사용자가 최종 구매한 상품은 과거에(일자 기준) 클릭했던 상품을 다시 찾아 구매한 비중이 어느정도 일까?


In [ ]:
# 클릭로그와 구매로그 건수
len(click_log[['user_id','product_id','dt']].drop_duplicates()), len(purchase_log[['user_id','product_id','dt']].drop_duplicates())

In [ ]:
# 구매로그와 클릭로그를 user_id 기준 join
purchase_click_merge = pd.merge(purchase_log, click_log, how = 'inner', on = ('user_id'))
purchase_click_merge

In [ ]:
# 컬럼명 변경
purchase_click_merge.columns = ['user_id','purchase_product_id','purchase_measure','purchase_dt','click_product_id','click_measure','click_dt']
purchase_click_merge

In [ ]:
# 상품을 구매하기 이전에(일자 기준) 클릭했던 상품을 다시 찾아 구매한건
purchase_bf_click = purchase_click_merge[ (purchase_click_merge['purchase_dt']>purchase_click_merge['click_dt']) & (purchase_click_merge['purchase_product_id']==purchase_click_merge['click_product_id']) ]

In [ ]:
purchase_bf_click

In [ ]:
# 상품을 구매하기 이전에 클릭했던 상품을 구매한 로그수, 전체 구매로그 중 비율
len(purchase_bf_click), len(purchase_bf_click)/len(purchase_log)

## 2) 사용자가 최종 구매한 상품은 최초 클릭 ~ 최종 구매 의사결정까지 얼마나 소요되는가?

In [ ]:
# 상품을 구매한 당일 포함하여, 최종 구매한 상품의 이전 클릭 데이터
purchase_ic_day_bf_click = purchase_click_merge[ (purchase_click_merge['purchase_dt']>=purchase_click_merge['click_dt']) & (purchase_click_merge['purchase_product_id']==purchase_click_merge['click_product_id']) ]
purchase_ic_day_bf_click

In [ ]:
len(purchase_ic_day_bf_click)

In [ ]:
# 최종 구매한 상품의 최초 클릭일
purchase_ic_day_bf_click = purchase_ic_day_bf_click.groupby(['user_id','purchase_product_id','purchase_dt'])['click_dt'].min().reset_index()
purchase_ic_day_bf_click

In [ ]:
# datetime 형태로 변환
purchase_ic_day_bf_click['purchase_dt'] = pd.to_datetime(purchase_ic_day_bf_click['purchase_dt'], format='%Y%m%d')
purchase_ic_day_bf_click['click_dt'] = pd.to_datetime(purchase_ic_day_bf_click['click_dt'], format='%Y%m%d')
purchase_ic_day_bf_click.info()

In [ ]:
# 최초 클릭 후 구매까지 구매의사결정 소요일수
purchase_ic_day_bf_click['click_to_purchase_days'] = (purchase_ic_day_bf_click['purchase_dt'] - purchase_ic_day_bf_click['click_dt']).dt.days
purchase_ic_day_bf_click

In [ ]:
# 클릭 후 구매까지 구매의사결정 소요일수 분포
fig=plt.figure(figsize=(10,5))
ax=fig.gca()
sns.histplot(purchase_ic_day_bf_click['click_to_purchase_days'], kde=True, discrete=True ,color='green')
plt.ylabel('purchase_log_count')
plt.xticks(range(min(purchase_ic_day_bf_click['click_to_purchase_days']),max(purchase_ic_day_bf_click['click_to_purchase_days'])+1 ))

In [ ]:
# 당일 최초 클릭 후 당일 구매인 0 건수
len(purchase_ic_day_bf_click[purchase_ic_day_bf_click['click_to_purchase_days']==0])

## 3) 성별/연령대별로 구매하는 상품의 카테고리 특성이 다를까?

In [ ]:
# 구매로그에 상품 카테고리 정보 맵핑
purchase_category = pd.merge(purchase_log, products[['product_id','category_kor','price']], how = 'inner', on = 'product_id')
purchase_category

In [ ]:
# 구매로그에 사용자 demo 정보 맵핑
purchase_category_demo = pd.merge(purchase_category, users, how = 'inner', on = 'user_id')
purchase_category_demo

In [ ]:
# demo별 분포
purchase_category_demo.groupby(['age_range','gender'])['user_id'].count()
pd.pivot_table(purchase_category_demo, index = 'age_range', columns = 'gender', values='user_id', aggfunc = 'count')

In [ ]:
purchase_category_age_cnt = purchase_category_demo.groupby('age_range')['user_id'].count()
fig=plt.figure(figsize=(10,5))
ax=fig.gca()
sns.barplot(x=purchase_category_age_cnt.index, y=purchase_category_age_cnt.values, color='green')
plt.ylabel('purchase_log_count')

In [ ]:
# 연령 regrouping
def age_regrouping(x):
    if x == '20-24' or x == '25-29' :
        return '20-29'
    elif x == '60-64' or x == '65-69' or x == '70-' :
        return '60-'
    else :
        return x

In [ ]:
purchase_category_demo['age_regroup'] = purchase_category_demo['age_range'].apply(lambda x : age_regrouping(x))
purchase_category_demo

In [ ]:
# demo별 카테고리별 구매건수, 구매량 집계
purchase_category_demo_stats = purchase_category_demo.groupby(['age_regroup','gender','category_kor']).agg({'measure':['count','sum'],'price':'mean'}).reset_index()
purchase_category_demo_stats

In [ ]:
purchase_category_demo_stats.columns = ['age_regroup','gender','category_kor','count','sum','price_mean']
purchase_category_demo_stats

In [ ]:
# demo별 구매건수와 구매량이 높은순으로 rank 부여 > 어떤 카테고리를 많이 구매했는지
purchase_category_demo_stats['rank'] = purchase_category_demo_stats.sort_values(['count','sum'], ascending = [False,False]).groupby(['age_regroup','gender']).cumcount()+1 # 그룹내 번호부여
purchase_category_demo_stats = purchase_category_demo_stats[purchase_category_demo_stats['age_regroup']!='15-19']

In [ ]:
# demo 별 1,2순위 카테고리 도출
purchase_category_demo_stats[purchase_category_demo_stats['rank']==2].sort_values(by=['age_regroup','gender','rank'], ascending=True)

In [ ]:
# demo 별 1,2순위 카테고리 도출
purchase_category_demo_stats[purchase_category_demo_stats['rank']==2].sort_values(by=['age_regroup','gender','rank'], ascending=True)

In [ ]:
# 1순위가 스마트폰,태블릿 스마트폰 데이터 price_mean 확인 목적
temp1 = purchase_category_demo_stats[(purchase_category_demo_stats['rank']==1) & (purchase_category_demo_stats['category_kor']=='스마트 폰, 태블릿 스마트 폰')]
temp1

In [ ]:
# 2순위가 스마트폰,태블릿 스마트폰 데이터 price_mean 확인 목적
temp2 = purchase_category_demo_stats[(purchase_category_demo_stats['rank']==2) & (purchase_category_demo_stats['category_kor']=='다이어트, 건강 건강 식품')]
temp2

In [ ]:
# 2순위가 스마트폰,태블릿 스마트폰 데이터 price_mean 확인 목적
temp3 = purchase_category_demo_stats[(purchase_category_demo_stats['rank']==2) & (purchase_category_demo_stats['category_kor']=='다이어트, 건강 개호 용품')]
temp3

In [ ]:
purchase_category_demo

In [ ]:
# demo별 카테고리별 평균구매금액
purchase_category_demo_price = purchase_category_demo.groupby(['age_regroup','gender','category_kor']).agg({'measure':['count','sum']}).reset_index()
purchase_category_demo_stats

# 상품 추천 로직

## 1) IBCF 기반 : 사용자가 클릭한 상품과 유사도가 높지만, 아직 클릭해보지 않은 상품 추천

In [ ]:
# 추천 대상 사용자
len(_20230401_users)

In [ ]:
# 추천대상자가 클릭한 상품 410개 - maxtrix allign
recomm_user_click = pd.merge(_20230401_users,click_log, how='inner', on='user_id')
len(recomm_user_click['product_id'].drop_duplicates()), len(recomm_user_click['user_id'].drop_duplicates())

In [ ]:
 # 410개 상품 클릭한 로그만 필터링 - maxtrix allign
 recommend_click_log = pd.merge(click_log,recomm_user_click['product_id'].drop_duplicates(),how='inner',on='product_id')
 recommend_click_log

In [ ]:
# 상품당 클릭한 유저수, 평균적으로 몇명의 사용자가 클릭하는지 > 너무 적은 사용자만 클릭한 상품 제외 고려
click_user_cnt = recommend_click_log.groupby('product_id')['user_id'].nunique().reset_index()
click_user_cnt['user_id'].mean(), click_user_cnt['user_id'].median()

In [ ]:
# 3명 이상이 클릭한 상품 데이터로 matrix 구성
click_log_user_10up = pd.merge(recommend_click_log, click_user_cnt[click_user_cnt['user_id']>=3]['product_id'], how = 'inner', on = 'product_id')
len(click_log_user_10up)

In [ ]:
click_log_user_10up['user_id'].nunique(), click_log_user_10up['product_id'].nunique()

In [ ]:
# product*user_id full matrix 65 * 3150
click_matrix = click_log_user_10up.pivot_table(index = 'product_id', columns = 'user_id', values='measure', aggfunc='sum')
click_matrix

In [ ]:
click_matrix_dummy = click_matrix.copy().fillna(0)
click_matrix_dummy

In [ ]:
pd.set_option('display.max_rows',20)
pd.set_option('display.max_columns',20)

In [ ]:
# cosine 유사도
item_similarity = cosine_similarity(click_matrix_dummy, click_matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=click_matrix.index, columns=click_matrix.index)

In [ ]:
item_similarity

In [ ]:
item_similarity #.head(20)

In [ ]:
products[products['product_id'] == 6082]

In [ ]:
products[products['product_id'] == 766]

In [ ]:
products[products['product_id'] == 309]

In [ ]:
products[products['product_id'] == 14]

In [ ]:
# 추천대상자 matrix
recomm_user_click_product = pd.merge(recomm_user_click, click_log_user_10up['product_id'].drop_duplicates(), how = 'inner', on='product_id' )
user_matrix = pd.pivot_table(recomm_user_click_product, index='user_id',columns='product_id', values='measure', aggfunc='sum')
user_matrix = user_matrix.copy().fillna(0)
user_matrix

In [ ]:
# 예측
def predict(ratings, similarity):
    pred = ratings.dot(similarity) / np.array([np.abs(similarity.sum(axis=1))])
    return pred

In [ ]:
item_prediction = predict(user_matrix, item_similarity)
item_prediction

In [ ]:
# 이미 클릭한 것은 제외
def remove_rated_items(ratings, predictions):
    rated_items = (ratings != 0)
    predictions[rated_items] = -np.inf
    return predictions

updated_predictions = remove_rated_items(user_matrix, item_prediction)

updated_predictions

In [ ]:
# top3

ibcf_recomm_temp = updated_predictions.apply(lambda row: pd.Series(row.nlargest(3).index), axis=1)
ibcf_recomm_temp.columns = ['product_id_1', 'product_id_2', 'product_id_3']

ibcf_recomm_temp = ibcf_recomm_temp.reset_index()
ibcf_recomm_temp

## 2) DEMO 기반 : 사용자와 동일한 성별/연령대 그룹이 가장 많이 구매한 상품 추천

In [ ]:
# 연령 regrouping
def age_regrouping(x):
    if  x == '-14' or x == '15-19' :
        return '-19'
    elif x == '20-24' or x == '25-29' :
        return '20-29'
    elif x == '30-34' or x == '35-39' :
        return '30-39'
    elif x == '40-44' or x == '45-49' :
        return '40-49'
    elif x == '50-54' or x == '55-59' :
        return '50-59'
    elif x == '60-64' or x == '65-69' or x == '70-' :
        return '60-'
    else :
        return x

In [ ]:
purchase_category_demo

In [ ]:
# 연령 10세 단위로 크게 regrouping
purchase_category_demo['age_regroup'] = purchase_category_demo['age_range'].apply(lambda x : age_regrouping(x))

In [ ]:
purchase_category_demo['age_regroup'].value_counts()

In [ ]:
# 연령/성별로 구매한 상품id별 구매건/구매량 집계
purchase_category_demo_product_cnt = purchase_category_demo.groupby(['age_regroup','gender','product_id']).agg({'measure' : ['count','sum']}).reset_index()
purchase_category_demo_product_cnt['age_regroup'].value_counts()
purchase_category_demo_product_cnt.columns = ['age_regroup','gender','product_id','count','sum']

In [ ]:
# 성별/연령별로 구매건,구매량이 많은 순으로 rank 부여
purchase_category_demo_product_cnt['rank'] = purchase_category_demo_product_cnt.sort_values(['count','sum'], ascending = [False,False]).groupby(['age_regroup','gender']).cumcount()+1 # 그룹내 번호부여

In [ ]:
pd.set_option('display.max_rows',10)
pd.set_option('display.max_columns',10)

In [ ]:
# 성별/연령별로 top 2만 확인 > 1위는 분명한데 2위부터 건수가 작아 애매함
purchase_category_demo_product_cnt[ (purchase_category_demo_product_cnt['rank']<=2)].sort_values(['age_regroup','gender','rank'], ascending = [True,True,True])

In [ ]:
# 성별/연령별로 구매 가장 많이한 top1 상품
demo_purchase_top1 = purchase_category_demo_product_cnt[ (purchase_category_demo_product_cnt['rank']==1)].sort_values(['age_regroup','gender','rank'], ascending = [True,True,True])
demo_purchase_top1

In [ ]:
# 성별/연령별로 구매 가장 많이한 top2 상품
demo_purchase_top2 = purchase_category_demo_product_cnt[ (purchase_category_demo_product_cnt['rank']==2)].sort_values(['age_regroup','gender','rank'], ascending = [True,True,True])
demo_purchase_top2

In [ ]:
# 성별/연령별로 구매 가장 많이한 top3 상품
demo_purchase_top3 = purchase_category_demo_product_cnt[ (purchase_category_demo_product_cnt['rank']==3)].sort_values(['age_regroup','gender','rank'], ascending = [True,True,True])
demo_purchase_top3

In [ ]:
# 그 1순위 상품은 무엇인가
pd.merge(demo_purchase_top1,products, how='inner', on = 'product_id').sort_values(by= ['age_regroup','gender'])

In [ ]:
# 그 2순위 상품은 무엇인가
pd.merge(demo_purchase_top2,products, how='inner', on = 'product_id').sort_values(by= ['age_regroup','gender'])

In [ ]:
# 그 3순위 상품은 무엇인가
pd.merge(demo_purchase_top3,products, how='inner', on = 'product_id').sort_values(by= ['age_regroup','gender'])

In [ ]:
demo_top2 = pd.merge( demo_purchase_top1[['age_regroup','gender','product_id','count','sum']],demo_purchase_top2[['age_regroup','gender','product_id','count','sum']], how = 'inner', on = ['age_regroup','gender'] )
demo_top2.columns = ['age_regroup','gender','product_id_1','count_1','sum_1','product_id_2','count_2','sum_2']
demo_top2

In [ ]:
pd.set_option('display.max_columns',20)
demo_top3 = pd.merge( demo_top2,demo_purchase_top3[['age_regroup','gender','product_id','count','sum']], how = 'inner', on = ['age_regroup','gender'] )
demo_top3.columns = ['age_regroup','gender','product_id_1','count_1','sum_1','product_id_2','count_2','sum_2','product_id_3','count_3','sum_3']
demo_top3

In [ ]:
# ibcf로 추천받지 않은 대상자
not_ibcf_recom_user = _20230401_users[~_20230401_users['user_id'].isin(ibcf_recomm_temp['user_id'])]
not_ibcf_recom_user['age_regroup'] = not_ibcf_recom_user['age_range'].apply(lambda x : age_regrouping(x))
not_ibcf_recom_user['age_regroup'].value_counts()

In [ ]:
len(ibcf_recomm_temp)

In [ ]:
len(not_ibcf_recom_user)

In [ ]:
_20230401_users['gender'].value_counts()

In [ ]:
# ibcf로 추천받지 않은 대상자에게 demo 기반 추천 맵핑 > 7명 미맵핑
demo_recom_temp = pd.merge(not_ibcf_recom_user, demo_top3[['age_regroup','gender','product_id_1','product_id_2','product_id_3']], how = 'inner', on = ['age_regroup','gender'])
len(demo_recom_temp)

In [ ]:
demo_recom_temp.info()

In [ ]:
# 기존에 클릭했던 상품인지 check하여 추천여부 결정
def recomm_yn(table1,table2) :
    joined_df_1 = pd.merge(table1,table2[['user_id','product_id']].drop_duplicates() ,how = 'inner', left_on = ['user_id','product_id_1'], right_on = ['user_id','product_id'] )

    joined_df_2 = pd.merge(table1,table2[['user_id','product_id']].drop_duplicates(),how = 'inner', left_on = ['user_id','product_id_2'], right_on = ['user_id','product_id'] )

    joined_df_3 = pd.merge(table1,table2[['user_id','product_id']].drop_duplicates(),how = 'inner', left_on = ['user_id','product_id_3'], right_on = ['user_id','product_id'] )

    table1['recomm_1'] = 1
    table1['recomm_2'] = 1
    table1['recomm_3'] = 1

    # 이미 클릭해본 상품인 경우 추천=-1으로 업데이트
    table1.loc[table1['user_id'].isin(joined_df_1['user_id']), 'recomm_1'] = np.nan
    table1.loc[table1['user_id'].isin(joined_df_2['user_id']), 'recomm_2'] = np.nan
    table1.loc[table1['user_id'].isin(joined_df_3['user_id']), 'recomm_3'] = np.nan

In [ ]:
pd.merge(demo_recom_temp,recomm_user_click[['user_id','product_id']].drop_duplicates() ,how = 'inner', left_on = ['user_id','product_id_1'], right_on = ['user_id','product_id'] )

In [ ]:
# demo기반 추천 중 이미 클릭한 상품은 제거
recomm_yn(demo_recom_temp,recomm_user_click)

In [ ]:
# 3개 모두 클릭했던 상품인지
demo_recom_temp['recomm_null'] = demo_recom_temp['recomm_1'].fillna(0)+ demo_recom_temp['recomm_2'].fillna(0) + demo_recom_temp['recomm_3'].fillna(0)
demo_recom_temp.head(20)

In [ ]:
# 3개 다 이미 클릭해서recomm_null==0 추천 불가능한 경우는 없음
demo_recom_temp['recomm_null'].value_counts()

In [ ]:
# user_id 8798	에게 7767이 추천됐지만, 이미 구매한 상품으로 recomm_1 == 0 적용
demo_recom_temp[demo_recom_temp['recomm_null']==2]

In [ ]:
demo_recom_temp['product_id_1'] = demo_recom_temp['product_id_1'] * demo_recom_temp['recomm_1']
demo_recom_temp['product_id_2'] = demo_recom_temp['product_id_2'] * demo_recom_temp['recomm_2']
demo_recom_temp['product_id_3'] = demo_recom_temp['product_id_3'] * demo_recom_temp['recomm_3']
demo_recom_temp

In [ ]:
demo_recom_temp[demo_recom_temp['recomm_null']==2]

In [ ]:
# nan인 경우 1순위 > 2순위 > 3순위 당기는 작업
demo_recom_temp['product_id_1'] = demo_recom_temp['product_id_1'].combine_first(demo_recom_temp['product_id_2']).combine_first(demo_recom_temp['product_id_3'])

In [ ]:
demo_recom_temp[demo_recom_temp['recomm_null']==2]

In [ ]:
final_recomm = pd.concat([ibcf_recomm_temp, demo_recom_temp[['user_id','product_id_1','product_id_2','product_id_3']]],ignore_index=True)

In [ ]:
len(ibcf_recomm_temp) ,len(demo_recom_temp)

In [ ]:
final_recomm.info()

In [ ]:
_20230401_users_recomm_final = pd.merge(_20230401_users,final_recomm[['user_id','product_id_1']],how='left',on = 'user_id')

In [ ]:
#연령/성별 이슈로 추천이 붙지 않은 7명
_20230401_users_recomm_final[_20230401_users_recomm_final['product_id_1'].isna()]

In [ ]:
# 전체 중 가장 많이 구매한 상품
top1 = purchase_log.groupby('product_id')['user_id'].count().reset_index().sort_values(by='user_id',ascending=False).head(1)['product_id']
top1

In [ ]:
# 전체 중 가장 많이 구매한 상품으로 추천하기
_20230401_users_recomm_final['product_id_1'].fillna(top1.item(),inplace=True)

In [ ]:
# 추천이 붙지 않은 case 없음
_20230401_users_recomm_final[_20230401_users_recomm_final['product_id_1'].isna()]

In [ ]:
# product_id_1 타입 int로 변환
_20230401_users_recomm_final['product_id_1'] = _20230401_users_recomm_final['product_id_1'].astype(int)
_20230401_users_recomm_final.head(20)

In [ ]:
len(_20230401_users_recomm_final)

In [ ]:
_20230401_users_recomm_final.to_csv('_20230401_users_recomm_final.csv',index=False)

In [ ]:
files.download('_20230401_users_recomm_final.csv')